# Chapter 2: Assembling and Running the Agent in Streamlit

## Learning Objectives

By the end of this chapter, you will:
- Integrate AI agent tools into a Streamlit application
- Create an intuitive user interface for resume-to-job matching
- Build a complete end-to-end AI-powered application
- Implement real-time feedback and progress indicators
- Deploy a professional-grade AI agent that users can actually use

## Introduction: Bringing It All Together

> **Instructor Cue:** Start with excitement: "This is the moment we've been building toward! We're going to create a complete AI application that anyone can use. Ask the audience: What would make this app truly useful and professional?"

In this final chapter, we're assembling all the pieces we've built throughout the workshop:

- **Data processing** from Module 1 & 2
- **Interactive dashboards** from Module 3  
- **AI agent tools** from Chapter 1

The result will be a polished, professional application that demonstrates the power of combining automation, data analysis, and AI.

### Our Final Product Features

- **Resume Input Interface**: Easy text area for pasting resumes
- **Real-time Skill Extraction**: See AI extract skills instantly
- **Intelligent Job Matching**: Advanced scoring and recommendations
- **Professional Results Display**: Beautiful presentation of findings
- **Multiple Match Options**: Not just one result, but top alternatives
- **Skill Market Analysis**: Understand demand for extracted skills

> **Instructor Cue:** Show a sketch or wireframe of the final app interface. This helps students visualize what they're building.

## Setting Up the Main Application

Let's create our final Streamlit application. Create a new file called `ai_job_agent_app.py` in the `04_module` directory.

> **Instructor Cue:** Have everyone create this new file. Explain that we're making a separate, focused app for the AI agent functionality.

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
%%writefile app/ai_job_agent_app.py

import streamlit as st
import pandas as pd
from datetime import datetime
import os

# Import our custom tools
from tools import (
    extract_skills_from_resume,
    find_best_job_match,
    get_skill_statistics,
    find_alternative_matches,
    validate_resume_text,
    safe_extract_skills
)

# Configure the page
st.set_page_config(
    page_title="AI Job Matching Agent",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        font-weight: bold;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }

    .skill-tag {
        display: inline-block;
        background-color: #e1f5fe;
        color: #01579b;
        padding: 0.25rem 0.75rem;
        margin: 0.25rem;
        border-radius: 1rem;
        font-size: 0.875rem;
        font-weight: 500;
    }

    .match-score {
        font-size: 1.5rem;
        font-weight: bold;
        color: #4caf50;
    }

    .job-card {
        border: 1px solid #e0e0e0;
        border-radius: 0.5rem;
        padding: 1rem;
        margin: 1rem 0;
        background-color: #fafafa;
    }
</style>
""", unsafe_allow_html=True)

# Main title
st.markdown('<h1 class="main-header">🤖 AI Job Matching Agent</h1>', unsafe_allow_html=True)
st.markdown('<p style="text-align: center; font-size: 1.2rem; color: #666;">Powered by AI • Find your perfect job match instantly</p>', unsafe_allow_html=True)

> **Instructor Cue:** Explain the custom CSS and how it makes the app look more professional. Point out that good UX design is crucial for user adoption.

## Loading Data and Environment Setup

Let's add robust data loading and environment checking:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

@st.cache_data
def load_job_data():
    """Load job data with error handling and user feedback"""
    try:
        df = pd.read_csv('data/indeed_jobs_combined.csv')

        # Basic data cleaning
        df = df.dropna(subset=['job_title', 'company_name'])

        # Ensure we have meaningful job descriptions
        df = df[df['job_description'].str.len() > 50]

        return df, None

    except FileNotFoundError:
        error_msg = "Job data file not found. Please ensure 'data/indeed_jobs_combined.csv' exists."
        return None, error_msg
    except Exception as e:
        error_msg = f"Error loading job data: {str(e)}"
        return None, error_msg

def check_environment():
    """Check if all required components are available"""
    issues = []

    # Check for API key
    if not os.getenv('GOOGLE_API_KEY'):
        issues.append("Google API key not found. Set GOOGLE_API_KEY environment variable.")

    # Check for tools module
    try:
        import tools
    except ImportError:
        issues.append("Tools module not found. Ensure tools.py is in the same directory.")

    return issues

# Initialize the application
def initialize_app():
    """Initialize the application with proper error handling"""

    # Check environment
    env_issues = check_environment()
    if env_issues:
        st.error("⚠️ Setup Issues Detected:")
        for issue in env_issues:
            st.error(f"• {issue}")

        st.info("💡 **Quick Setup Help:**")
        st.code("""
# 1. Install required packages using uv:
uv sync

# If you don't have uv installed:
pip install uv
uv sync

# 2. Get your Google API key from:
# https://aistudio.google.com/app/apikey

# 3. Set your Google API key (choose one method):

# Method 1: Environment variable (recommended)
export GOOGLE_API_KEY="your-api-key-here"

# Method 2: In your terminal
set GOOGLE_API_KEY=your-api-key-here

# 4. Then restart your Streamlit app
        """)
        return None, None

    # Load job data
    jobs_df, error = load_job_data()
    if error:
        st.error(f"❌ {error}")
        st.info("💡 Make sure you've copied the job data from previous modules.")
        return None, None

    st.success(f"✅ Loaded {len(jobs_df)} job postings successfully!")
    return jobs_df, None

# Initialize app
jobs_df, init_error = initialize_app()

if init_error or jobs_df is None:
    st.stop()

> **Instructor Cue:** Walk through the initialization process. Emphasize how proper error handling and user feedback make applications more professional and user-friendly.

## Building the User Interface

Now let's create the main user interface with clear sections:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

# Create the main layout
col1, col2 = st.columns([2, 1])

with col1:
    st.header("📄 Resume Input")

    # Option to load sample resume
    if st.button("📋 Load Sample Resume", type="secondary"):
        try:
            with open('data/sample_resume.txt', 'r') as file:
                st.session_state.resume_text = file.read()
        except FileNotFoundError:
            st.error("Sample resume file not found.")

    # Resume text input
    resume_text = st.text_area(
        "Paste your resume text here:",
        value=st.session_state.get('resume_text', ''),
        height=400,
        placeholder="Copy and paste the text content of your resume here...\n\nInclude:\n• Work experience\n• Technical skills\n• Education\n• Projects",
        help="The AI will analyze your resume to extract relevant skills and find matching jobs."
    )

    # Store in session state
    st.session_state.resume_text = resume_text

with col2:
    st.header("ℹ️ How It Works")

    st.markdown("""
    **Step 1:** 📄 Paste your resume text

    **Step 2:** 🧠 AI extracts your key skills

    **Step 3:** 🔍 Algorithm finds best job matches

    **Step 4:** 🎯 Get personalized recommendations

    ---

    **What we analyze:**
    • Programming languages
    • Frameworks & tools
    • Technical competencies
    • Experience level indicators
    """)

    # Progress indicator
    if 'processing_step' in st.session_state:
        st.info(f"🔄 {st.session_state.processing_step}")

> **Instructor Cue:** Explain how the two-column layout provides both functionality and guidance. The "How It Works" section helps users understand what to expect.

## Implementing the Core Processing Logic

Let's add the main processing workflow with clear user feedback:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

# Main processing section
st.header("🚀 Find Your Perfect Job Match")

# Validate input before processing
if resume_text and len(resume_text.strip()) > 20:

    # Validation feedback
    validation = validate_resume_text(resume_text)

    if validation['warnings']:
        with st.expander("⚠️ Resume Analysis Notes", expanded=False):
            for warning in validation['warnings']:
                st.warning(warning)

    # Processing button
    if st.button("🎯 Find My Best Job Matches", type="primary", use_container_width=True):

        # Create processing pipeline with progress tracking
        progress_bar = st.progress(0)
        status_text = st.empty()

        try:
            # Step 1: Extract skills
            status_text.text("🧠 Analyzing resume and extracting skills...")
            progress_bar.progress(25)

            skill_result = safe_extract_skills(resume_text)

            if not skill_result['success']:
                st.error("❌ Failed to extract skills from resume:")
                for issue in skill_result['issues']:
                    st.error(f"• {issue}")
                st.stop()

            skills = skill_result['skills']

            # Step 2: Analyze job market
            status_text.text("📊 Analyzing job market demand...")
            progress_bar.progress(50)

            skill_stats = get_skill_statistics(skills, jobs_df)

            # Step 3: Find matches
            status_text.text("🔍 Finding best job matches...")
            progress_bar.progress(75)

            best_match = find_best_job_match(skills, jobs_df)
            alternative_matches = find_alternative_matches(skills, jobs_df, top_n=5)

            # Step 4: Display results
            status_text.text("✅ Analysis complete!")
            progress_bar.progress(100)

            # Clear progress indicators
            progress_bar.empty()
            status_text.empty()

            # Store results in session state
            st.session_state.skills = skills
            st.session_state.skill_stats = skill_stats
            st.session_state.best_match = best_match
            st.session_state.alternative_matches = alternative_matches

        except Exception as e:
            st.error(f"❌ An error occurred during processing: {str(e)}")
            st.info("💡 This might be due to API rate limits or connectivity issues. Please try again.")

else:
    st.info("👆 Please paste your resume text above to get started.")

> **Instructor Cue:** Demonstrate the progress bar and status updates. Explain how this provides user feedback during potentially slow operations. Point out the error handling strategy.

## Creating Beautiful Results Display

Now let's create an impressive results display:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

# Display results if available
if 'skills' in st.session_state and 'best_match' in st.session_state:

    skills = st.session_state.skills
    skill_stats = st.session_state.skill_stats
    best_match = st.session_state.best_match
    alternative_matches = st.session_state.alternative_matches

    st.header("🎯 Your Job Match Results")

    # Skills section
    st.subheader("🛠️ Extracted Skills")

    # Display skills as tags
    skills_html = ""
    for skill in skills:
        skills_html += f'<span class="skill-tag">{skill}</span>'

    st.markdown(skills_html, unsafe_allow_html=True)

    # Skill demand analysis
    with st.expander("📊 Skill Market Demand Analysis", expanded=False):
        st.write("**Your skills in the job market:**")

        for skill, stats in skill_stats.items():
            col1, col2, col3 = st.columns([2, 1, 1])

            with col1:
                st.write(f"**{skill}**")
            with col2:
                st.write(f"{stats['jobs_mentioning']} jobs")
            with col3:
                demand_color = {
                    'Very High': '🟢',
                    'High': '🟡',
                    'Medium': '🟠',
                    'Low': '🔴',
                    'Very Low': '⚪'
                }.get(stats['demand_level'], '⚪')
                st.write(f"{demand_color} {stats['demand_level']}")

    st.markdown("---")

    # Best match section
    if "error" not in best_match:
        st.subheader("🥇 Your Best Job Match")

        # Match score display
        col1, col2, col3 = st.columns([2, 1, 1])

        with col1:
            st.markdown(f"### {best_match['job_title']}")
            st.markdown(f"**🏢 {best_match['company_name']}**")
            st.markdown(f"📍 {best_match['location']}")

        with col2:
            st.markdown(f'<div class="match-score">Match Score: {best_match["match_score"]}</div>', unsafe_allow_html=True)

        with col3:
            st.markdown(f"💰 **{best_match['salary']}**")
            st.markdown(f"🔍 *{best_match['total_jobs_analyzed']} jobs analyzed*")

        # Matched skills
        st.write("**🎯 Your matching skills for this role:**")
        matched_skills_html = ""
        for skill in best_match['matched_skills']:
            matched_skills_html += f'<span class="skill-tag" style="background-color: #c8e6c9;">{skill}</span>'
        st.markdown(matched_skills_html, unsafe_allow_html=True)

        # Job description preview
        with st.expander("📝 Job Description Preview", expanded=False):
            st.write(best_match['job_description'])

        # Alternative matches
        if alternative_matches and len(alternative_matches) > 1:
            st.subheader("🔄 Alternative Job Matches")

            for i, match in enumerate(alternative_matches[1:], 2):  # Skip first (best match)
                with st.expander(f"#{i}: {match['job_title']} at {match['company_name']} (Score: {match['match_score']})", expanded=False):

                    col1, col2 = st.columns(2)

                    with col1:
                        st.write(f"**Company:** {match['company_name']}")
                        st.write(f"**Location:** {match['location']}")
                        st.write(f"**Salary:** {match['salary']}")

                    with col2:
                        st.write(f"**Match Score:** {match['match_score']}")
                        st.write(f"**Matched Skills:** {', '.join(match['matched_skills'])}")

    else:
        st.error("❌ No suitable job matches found.")
        st.info("💡 Try updating your resume with more technical skills or specific technologies.")

> **Instructor Cue:** Walk through the results display step by step. Point out how the UI provides multiple levels of information - from high-level match to detailed alternatives. The color coding and icons make it visually appealing.

## Adding Advanced Features

Let's add some advanced features to make our app even more useful:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

# Advanced features section
st.header("📈 Advanced Features")

# Feature tabs
tab1, tab2, tab3 = st.tabs(["💾 Export Results", "🔧 Customize Search", "📚 Tips & Help"])

with tab1:
    st.subheader("Export Your Results")

    if 'best_match' in st.session_state and "error" not in st.session_state.best_match:

        # Generate export data
        export_data = {
            "analysis_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "extracted_skills": st.session_state.skills,
            "best_match": st.session_state.best_match,
            "alternative_matches": st.session_state.alternative_matches[:3],  # Top 3
            "skill_demand_analysis": st.session_state.skill_stats
        }

        # Create downloadable report
        import json
        json_report = json.dumps(export_data, indent=2)

        col1, col2 = st.columns(2)

        with col1:
            st.download_button(
                label="📥 Download JSON Report",
                data=json_report,
                file_name=f"job_match_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
                mime="application/json"
            )

        with col2:
            # Create a simple text summary
            text_summary = f"""
JOB MATCH ANALYSIS REPORT
Generated: {export_data['analysis_date']}

EXTRACTED SKILLS: {', '.join(st.session_state.skills)}

BEST MATCH:
- Position: {st.session_state.best_match['job_title']}
- Company: {st.session_state.best_match['company_name']}
- Location: {st.session_state.best_match['location']}
- Match Score: {st.session_state.best_match['match_score']}
- Matched Skills: {', '.join(st.session_state.best_match['matched_skills'])}

ALTERNATIVE MATCHES:
"""
            for i, match in enumerate(st.session_state.alternative_matches[:3], 1):
                text_summary += f"{i}. {match['job_title']} at {match['company_name']} (Score: {match['match_score']})\n"

            st.download_button(
                label="📄 Download Text Summary",
                data=text_summary,
                file_name=f"job_match_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt",
                mime="text/plain"
            )
    else:
        st.info("🔍 Run an analysis first to export results.")

with tab2:
    st.subheader("Customize Your Job Search")

    # Filters
    st.write("**Filter job matches by:**")

    # Location preference
    available_locations = jobs_df['location'].unique() if jobs_df is not None else []
    location_filter = st.multiselect(
        "Preferred locations:",
        options=available_locations,
        help="Leave empty to search all locations"
    )

    # Company size preference
    company_size = st.selectbox(
        "Company size preference:",
        options=["Any", "Startup", "Medium", "Large"],
        help="This would filter by company size if we had that data"
    )

    # Remote work preference
    remote_preference = st.selectbox(
        "Remote work preference:",
        options=["Any", "Remote Only", "Hybrid", "On-site Only"]
    )

    st.info("💡 **Coming Soon:** These filters will be applied to job matching in future versions!")

with tab3:
    st.subheader("Tips for Better Results")

    st.markdown("""
    **📝 Resume Writing Tips:**

    • **Include specific technologies**: Instead of "programming," list "Python, JavaScript, React"
    • **Mention frameworks**: Django, Flask, Express.js, etc.
    • **Add tools and platforms**: Git, Docker, AWS, etc.
    • **Include project details**: Brief descriptions of what you built
    • **Use industry keywords**: Terms commonly found in job postings

    **🎯 Getting Better Matches:**

    • **Be specific about experience level**: Junior, Senior, Lead, etc.
    • **Include soft skills**: Teamwork, communication, problem-solving
    • **Mention certifications**: AWS Certified, Google Analytics, etc.
    • **Add domain expertise**: E-commerce, healthcare, fintech, etc.

    **🚀 Improving Your Profile:**

    • **Learn in-demand skills**: Check the skill demand analysis above
    • **Build a portfolio**: GitHub projects showcasing your abilities
    • **Get certifications**: Validate your skills with recognized credentials
    • **Network actively**: Connect with professionals in your target field
    """)

> **Instructor Cue:** Explain how these advanced features add real value. The export functionality lets users save and share results, while the tips help them improve their profiles.

## Adding Sidebar Controls and Information

Let's add a comprehensive sidebar with controls and information:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

# Sidebar content
with st.sidebar:
    st.header("🔧 Controls")

    # Clear results button
    if st.button("🔄 Clear All Results", type="secondary"):
        for key in ['skills', 'skill_stats', 'best_match', 'alternative_matches', 'resume_text']:
            if key in st.session_state:
                del st.session_state[key]
        st.experimental_rerun()

    st.markdown("---")

    # App information
    st.header("ℹ️ About This App")

    st.markdown("""
    **AI Job Matching Agent** uses advanced AI to:

    🧠 **Analyze** your resume content

    🔍 **Extract** relevant technical skills

    📊 **Score** job matches algorithmically

    🎯 **Recommend** best opportunities

    ---

    **Technologies Used:**
    • Google Gemini 2.5 Flash for text analysis
    • PydanticAI for agent framework
    • Streamlit for web interface
    • Pandas for data processing
    """)

    # Statistics
    if jobs_df is not None:
        st.subheader("📊 Database Stats")
        st.metric("Total Jobs", len(jobs_df))
        st.metric("Companies", jobs_df['company_name'].nunique())
        st.metric("Locations", jobs_df['location'].nunique())

    st.markdown("---")

    # Help section
    st.subheader("❓ Need Help?")

    with st.expander("API Setup"):
        st.code("""
# 1. Get your free API key from:
# https://aistudio.google.com/app/apikey

# 2. Set your Google API key:
export GOOGLE_API_KEY="your-key-here"

# Or create a .env file:
GOOGLE_API_KEY=your-key-here

# Gemini 2.5 Flash offers generous free limits!
        """)

    with st.expander("Troubleshooting"):
        st.markdown("""
        **Common Issues:**

        • **No skills extracted**: Resume might be too short or unstructured
        • **No job matches**: Try including more technical skills
        • **API errors**: Check your Google API key and internet connection
        • **Slow performance**: Large resumes take longer to process
        """)

    # Footer
    st.markdown("---")
    st.markdown("""
    <div style='text-align: center; font-size: 0.8rem; color: #666;'>
    🤖 AI Job Agent v1.0<br>
    Built with ❤️ using Python
    </div>
    """, unsafe_allow_html=True)

> **Instructor Cue:** Explain how the sidebar provides essential controls and information without cluttering the main interface. This is professional UX design.

## Error Handling and Performance Optimization

Let's add comprehensive error handling and performance optimizations:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

# Performance and caching optimizations
@st.cache_data(ttl=3600)  # Cache for 1 hour
def cached_skill_extraction(resume_text: str) -> dict[str, Any]:
    """Cache skill extraction to avoid repeated API calls"""
    return safe_extract_skills(resume_text)

@st.cache_data
def cached_job_analysis(skills_tuple: tuple, jobs_data_hash: str) -> dict[str, Any]:
    """Cache job analysis results"""
    skills = list(skills_tuple)  # Convert back from tuple (needed for caching)

    best_match = find_best_job_match(skills, jobs_df)
    alternative_matches = find_alternative_matches(skills, jobs_df, top_n=5)
    skill_stats = get_skill_statistics(skills, jobs_df)

    return {
        'best_match': best_match,
        'alternative_matches': alternative_matches,
        'skill_stats': skill_stats
    }

# Function to create a hash of the jobs dataframe for caching
@st.cache_data
def get_jobs_hash(df: pd.DataFrame) -> str:
    """Create a hash of the jobs dataframe for cache invalidation"""
    return str(hash(str(df.values.tobytes())))

# Error boundary wrapper
def safe_execute(func, *args, **kwargs):
    """Safely execute a function with error handling"""
    try:
        return func(*args, **kwargs), None
    except Exception as e:
        error_msg = f"Error in {func.__name__}: {str(e)}"
        return None, error_msg

# Add performance monitoring
if 'page_loads' not in st.session_state:
    st.session_state.page_loads = 0

st.session_state.page_loads += 1

# Show debug info for instructors
if st.checkbox("🔧 Show Debug Info", value=False):
    st.subheader("Debug Information")

    debug_info = {
        "Page loads this session": st.session_state.page_loads,
        "Session state keys": list(st.session_state.keys()),
        "Jobs dataframe shape": jobs_df.shape if jobs_df is not None else "Not loaded",
        "Google API key set": bool(os.getenv('GOOGLE_API_KEY')),
        "Cache info": st.cache_data.get_stats() if hasattr(st.cache_data, 'get_stats') else "Not available"
    }

    for key, value in debug_info.items():
        st.write(f"**{key}:** {value}")

> **Instructor Cue:** Explain how caching improves performance and reduces API costs. The debug section helps with troubleshooting during development.

## Testing and Validation

Let's add built-in testing capabilities:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

# Testing section (only show in development mode)
if st.checkbox("🧪 Developer Mode", value=False):
    st.header("🧪 Testing & Validation")

    tab1, tab2 = st.tabs(["Component Tests", "Sample Data"])

    with tab1:
        st.subheader("Test Individual Components")

        if st.button("Test Skill Extraction"):
            test_resume = "Python developer with experience in Django, React, and AWS."
            result = safe_extract_skills(test_resume)
            st.json(result)

        if st.button("Test Job Matching"):
            test_skills = ["Python", "Django", "React"]
            if jobs_df is not None:
                result = find_best_job_match(test_skills, jobs_df)
                st.json(result)

        if st.button("Test Skill Statistics"):
            test_skills = ["Python", "JavaScript", "SQL"]
            if jobs_df is not None:
                result = get_skill_statistics(test_skills, jobs_df)
                st.json(result)

    with tab2:
        st.subheader("Sample Data Preview")

        if jobs_df is not None:
            st.write("**Jobs Data Sample:**")
            st.dataframe(jobs_df.head())

            st.write("**Data Quality Check:**")
            st.write(f"- Total rows: {len(jobs_df)}")
            st.write(f"- Missing job titles: {jobs_df['job_title'].isna().sum()}")
            st.write(f"- Missing descriptions: {jobs_df['job_description'].isna().sum()}")
            st.write(f"- Average description length: {jobs_df['job_description'].str.len().mean():.0f} characters")

> **Instructor Cue:** This testing section is valuable for debugging and demonstrating how the components work independently. It's also educational for students to see how to test their code.

## Final Application Assembly

Here's how to run the complete application:

In [ ]:
%%writefile -a app/ai_job_agent_app.py

# Main execution flow
def main():
    """Main application flow"""

    # Show welcome message for first-time users
    if st.session_state.page_loads == 1:
        st.balloons()
        st.success("🎉 Welcome to the AI Job Matching Agent! Paste your resume above to get started.")

if __name__ == "__main__":
    main()

> **Instructor Cue:** Save the complete file and demonstrate running it. Show how all the pieces work together seamlessly.

## Running Your Complete AI Agent

To run your complete AI agent, make sure you have both files:

1. `tools.py` (from Chapter 1)
2. `ai_job_agent_app.py` (from this chapter)

> **Instructor Cue:** Have everyone run their complete application. Walk around and help with any final issues. This is the culmination of the entire workshop!

## Exercise: Test and Enhance Your Agent

> **Instructor Cue:** Give participants 20 minutes to test and enhance their application:

Try these enhancements:

1. **Test with Different Resumes:**
   - Use your own resume
   - Try resumes from different fields
   - Test with very technical vs. general resumes

2. **Improve the UI:**
   - Add more custom CSS styling
   - Create additional information sections
   - Improve the color scheme or layout

3. **Enhance the Algorithm:**
   - Modify the scoring in `calculate_job_score()`
   - Add new matching criteria
   - Implement weighted skill matching

4. **Add New Features:**
   - Salary range filtering
   - Location-based matching
   - Company size preferences

## Deployment Considerations

> **Instructor Cue:** Discuss real-world deployment options:

For deploying your AI agent to production: Streamlit Cloud (Easiest)

## Best Practices for Production

> **Instructor Cue:** Share production-ready practices:

1. **Security:**
   - Never hardcode API keys
   - Use environment variables or secret management
   - Implement rate limiting for API calls

2. **Performance:**
   - Cache expensive operations
   - Implement pagination for large datasets
   - Monitor API usage and costs

3. **User Experience:**
   - Add loading states and progress indicators
   - Provide clear error messages
   - Include help and documentation

4. **Monitoring:**
   - Log user interactions and errors
   - Monitor API costs and usage
   - Track application performance

## Troubleshooting Common Issues

> **Instructor Cue:** Keep this handy for final troubleshooting:

**Application Won't Start:**
- Check that both `tools.py` and `ai_job_agent.py` are in the same directory
- Verify all imports are working
- Ensure required packages are installed

**API Errors:**
- Verify Google API key is set correctly
- Check internet connection
- Monitor API rate limits and usage

**No Skills Extracted:**
- Resume text might be too short or poorly formatted
- Try the fallback extraction method
- Check for special characters or encoding issues

**Poor Job Matches:**
- Verify job data is loaded correctly
- Check that extracted skills are relevant
- Adjust scoring algorithm if needed

**Performance Issues:**
- Use caching for repeated operations
- Consider pagination for large datasets
- Monitor memory usage with large files

## Key Achievements

> **Instructor Cue:** Celebrate what they've accomplished:

🎉 **Congratulations! You've built a complete AI-powered application that:**

- **Processes natural language** using advanced AI models
- **Extracts meaningful insights** from unstructured text
- **Performs intelligent matching** using custom algorithms
- **Provides professional user experience** with modern web interface
- **Handles real-world data** with robust error handling
- **Scales for production use** with caching and optimization

## Real-World Applications

> **Instructor Cue:** Connect to broader applications:

The techniques you've learned can be applied to:

- **Resume screening systems** for HR departments
- **Candidate matching platforms** for recruiting agencies
- **Skill gap analysis tools** for career development
- **Job recommendation engines** for job boards
- **Academic program matching** for students
- **Freelancer-project matching** for gig platforms

## Workshop Recap: Full Journey

> **Instructor Cue:** Recap the entire workshop journey:

**Module 1:** Built automation scripts and data collection systems
**Module 2:** Processed and cleaned real-world data with AI assistance  
**Module 3:** Created interactive dashboards and reporting tools
**Module 4:** Assembled everything into an intelligent AI agent

**You now have:**
- A complete toolkit for data-driven automation
- Experience with modern AI integration
- Skills in building production-ready applications
- A portfolio piece that demonstrates real capabilities

## Next Steps and Continued Learning

> **Instructor Cue:** Provide guidance for continued learning:

**Immediate Next Steps:**
1. Deploy your application to Streamlit Cloud
2. Share it with friends and colleagues for feedback
3. Add it to your portfolio and LinkedIn profile

**Continued Learning:**
- **Advanced AI Integration**: Learn about fine-tuning, embeddings, and RAG systems
- **Production Deployment**: Docker, Kubernetes, cloud platforms
- **Data Engineering**: Building robust data pipelines
- **Machine Learning**: Moving beyond AI APIs to custom models

**Stay Connected:**
- Join AI and data science communities
- Follow industry blogs and research
- Build more projects using these techniques
- Consider advanced courses in AI/ML engineering

## Final Challenge

> **Instructor Cue:** Give them a stretch goal:

**Take-Home Challenge:** Adapt your AI agent for a different domain:

- **Academic Program Matcher**: Match student profiles to university programs
- **Freelancer Project Matcher**: Connect freelancers with suitable gigs
- **Investment Opportunity Finder**: Match investor profiles with startups
- **Medical Specialist Referral**: Match patient symptoms with appropriate specialists

Use the same framework but adapt the data sources, extraction logic, and matching algorithms.

## Conclusion

> **Instructor Cue:** End on an inspiring note:

You've completed an incredible journey from automation scripts to AI-powered applications. The skills you've learned - combining automation, data analysis, visualization, and AI - represent the future of software development.

Keep building, keep learning, and most importantly, keep solving real problems with technology. The world needs more developers who can bridge the gap between data and intelligence, between automation and insight.

**Thank you for an amazing workshop! 🚀**